# Notebook 4: Knowledge Base Assistant

**User Stories**: US4 (Natural-Language CRM Query) + US3 (KB Search)  
**Persona**: Customer Service Representative (CSR)  

Ask natural-language questions against the Salesforce Knowledge Base
and get accurate answers with article citations.

In [ ]:
# Cell 2: Environment + Auth Setup
import os
from pathlib import Path

from dotenv import load_dotenv

env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)

required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing required environment variables: {missing}")

print("Environment configured successfully.")
print(f"  Azure AI endpoint: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:40]}...")
print(f"  Salesforce instance: {os.environ['SF_INSTANCE_URL']}")

In [ ]:
# Cell 3: Start MCP Server + Connect with MCP Client
import asyncio
import subprocess
import sys
import time

import nest_asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client

nest_asyncio.apply()

project_root = os.path.abspath("..")

KB_PORT = 8104
kb_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_knowledge.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(KB_PORT)},
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
time.sleep(3)

if kb_process.poll() is not None:
    stderr = kb_process.stderr.read().decode() if kb_process.stderr else ""
    raise RuntimeError(f"Knowledge MCP server failed to start: {stderr}")

MCP_URL = f"http://127.0.0.1:{KB_PORT}/sse"
print(f"✅ Knowledge MCP server started (PID: {kb_process.pid}) at {MCP_URL}")


async def _connect():
    _sse_cm = sse_client(url=MCP_URL)
    read_stream, write_stream = await _sse_cm.__aenter__()
    _sess_cm = ClientSession(read_stream, write_stream)
    session = await _sess_cm.__aenter__()
    await session.initialize()
    tools_result = await session.list_tools()
    return _sse_cm, _sess_cm, session, tools_result.tools

_sse_cm, _sess_cm, mcp_session, mcp_tools = asyncio.get_event_loop().run_until_complete(_connect())

print(f"✅ MCP session connected — {len(mcp_tools)} tools available:")
for t in mcp_tools:
    print(f"   • {t.name}: {t.description[:80] if t.description else ''}")

In [ ]:
# Cell 4: Create OpenAI Client + System Prompt + Chat Helper
import asyncio
import json
from pathlib import Path as _Path

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)
client = project_client.get_openai_client()

system_prompt = _Path("../agents/service/system_prompt.md").read_text(encoding="utf-8")
MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")

openai_tools = [
    {
        "type": "function",
        "function": {
            "name": t.name,
            "description": t.description or "",
            "parameters": t.inputSchema or {"type": "object", "properties": {}},
        },
    }
    for t in mcp_tools
]

async def _call_tool(name: str, arguments: dict) -> str:
    result = await mcp_session.call_tool(name, arguments)
    return "\n".join(b.text for b in result.content if hasattr(b, "text")) or ""

def chat(user_message: str, *, history: list | None = None, max_rounds: int = 10) -> list:
    if history is None:
        history = [{"role": "system", "content": system_prompt}]
    history.append({"role": "user", "content": user_message})
    loop = asyncio.get_event_loop()
    for _ in range(max_rounds):
        resp = client.chat.completions.create(model=MODEL, messages=history, tools=openai_tools, tool_choice="auto")
        msg = resp.choices[0].message
        history.append(msg)
        if not msg.tool_calls:
            break
        for tc in msg.tool_calls:
            args = json.loads(tc.function.arguments) if tc.function.arguments else {}
            tool_result = loop.run_until_complete(_call_tool(tc.function.name, args))
            history.append({"role": "tool", "tool_call_id": tc.id, "content": tool_result})
    return history

def show(history: list):
    from IPython.display import Markdown, display
    for m in reversed(history):
        content = m.content if isinstance(m, dict) else m.content
        role = m.get("role") if isinstance(m, dict) else getattr(m, "role", None)
        if role == "assistant" and content:
            display(Markdown(content))
            return

print(f"✅ OpenAI client ready (model: {MODEL})")
print(f"✅ {len(openai_tools)} MCP tools mapped to OpenAI functions")

In [ ]:
# Cell 5: KB Query — "How do I reset a customer's API key?"

history = chat("How do I reset a customer's API key?")
show(history)

In [ ]:
# Cell 6: Follow-up — "What about SSO configuration?"

history = chat("What about SSO configuration?", history=history)
show(history)

In [ ]:
# Cell 7: Cleanup — close MCP session + stop server
import asyncio


async def _cleanup():
    await _sess_cm.__aexit__(None, None, None)
    await _sse_cm.__aexit__(None, None, None)

asyncio.get_event_loop().run_until_complete(_cleanup())

kb_process.terminate()
kb_process.wait(timeout=5)
print(f"✅ MCP session closed & server stopped (PID: {kb_process.pid})")
print("\nKB assistant session complete.")